<a href="https://colab.research.google.com/github/dolmani38/Summary/blob/master/Korean_QA_on_Wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Building a QA System with BERT on Wikipedia

https://qa.fastforwardlabs.com/pytorch/hugging%20face/wikipedia/bert/transformers/2020/05/19/Getting_Started_with_QA.html

위의 내용을 한국어 QA로 변경

지식 base = wiki+네이버(view,kin,news)

In [1]:
!pip install transformers==3
!pip install wikipedia
!pip install sentence-transformers

     |████████████████████████████████| 757kB 4.2MB/s 
     |████████████████████████████████| 3.0MB 39.9MB/s 
     |████████████████████████████████| 890kB 36.9MB/s 
     |████████████████████████████████| 1.1MB 37.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=715b729b206719ef262d581f371acbafffeb5297980d4b7d365551378d08fb34
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11686 sha256=d0302eb491ffb99e2f12d854a0f78de2879dedf51901c26a936acb01d0748352
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia
     |████████████████████████████████| 71kB 3.5MB/s 
     |████████████████████████████████| 1.3MB 7.8MB/s 
     |████████████████████████████████| 2.9MB 38.8MB/s 
     |███████████████

##한국어 SQUAD
KorQuAD 2.0 사용 https://korquad.github.io/

##영어 + 한국어 BERT 모델의 사용

#Extending Sentence Embeddings Models to New Languages

**Available Pre-trained Models**

*   **distiluse-base-multilingual-cased**: Supported languages: Arabic, Chinese, Dutch, English, French, German, Italian, Korean, Polish, Portuguese, Russian, Spanish, Turkish. Model is based on DistilBERT-multi-lingual.
*   **xlm-r-base-en-ko-nli-ststb**: Supported languages: English, Korean. Performance on Korean STSbenchmark: 81.47
*   **xlm-r-large-en-ko-nli-ststb**: Supported languages: English, Korean. Performance on Korean STSbenchmark: 84.05 --> 이거 사용!


참조:https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/multilingual-models.md



In [2]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

SQUAD_MODEL = "monologg/koelectra-base-v3-finetuned-korquad"
STS_MODEL = "xlm-r-large-en-ko-nli-ststb"

tokenizer = AutoTokenizer.from_pretrained(SQUAD_MODEL)
model = AutoModelForQuestionAnswering.from_pretrained(SQUAD_MODEL)
#reader = DocumentReader(SQUAD_MODEL) 
embedder = SentenceTransformer(STS_MODEL)

100%|██████████| 1.80G/1.80G [05:38<00:00, 5.30MB/s]


In [37]:
class AnswerVoter:
  def __init__(self, threshold_score=3,max_rank=2):
    self.answes = {}
    self.threshold_score= threshold_score
    self.max_rank = max_rank

  def add_ans(self,ans,score,src=''):
    key = ans.replace(' ','')
    if score > self.threshold_score:
      print(' --- Candidate answer:',ans,score)
      if key in self.answes:
        self.answes[key][1] += score
        if src in self.answes[key][2]:
          pass
        else:
          self.answes[key][2].append(src)
      else:
        self.answes[key] = [ans,score,[src]]

  def get_ans(self):
    answer = []
    sorted_x = sorted(self.answes.items(), key=lambda kv: kv[1][1],reverse=True)
    for i in range(min(self.max_rank,len(sorted_x))):
      answer.append(sorted_x[i])

    return answer

  def print(self):
    answer = self.get_ans()
    for ans in answer:
      print('Answer:',ans[1][0], ' score:',ans[1][1], ' source:',ans[1][2])
      #print('Answer:',ans)

In [38]:
av = AnswerVoter()

av.add_ans('8명',4.2,'aaaaa')
av.add_ans('8 명',4.5,'bbbb')
av.add_ans('18명',3.5)
av.add_ans('18 명',3.1)
av.add_ans('118명',3.1)
av.add_ans('118 명',3.0)

#ans = av.get_ans()

av.print()

 --- Candidate answer: 8명 4.2
 --- Candidate answer: 8 명 4.5
 --- Candidate answer: 18명 3.5
 --- Candidate answer: 18 명 3.1
 --- Candidate answer: 118명 3.1
Answer: 8명  score: 8.7  source: ['aaaaa', 'bbbb']
Answer: 18명  score: 6.6  source: ['']


In [39]:
import wikipedia as wiki
import pprint as pp
from collections import OrderedDict

class DocumentReader:
    def __init__(self, _tokenizer, _model):
        #self.READER_PATH = pretrained_model_name_or_path
        self.tokenizer = _tokenizer #AutoTokenizer.from_pretrained(self.READER_PATH)
        self.model = _model #AutoModelForQuestionAnswering.from_pretrained(self.READER_PATH)
        self.max_len = self.model.config.max_position_embeddings
        self.chunked = False

    def tokenize(self, question, text):
        self.inputs = self.tokenizer.encode_plus(question, text, add_special_tokens=True, return_tensors="pt")
        self.input_ids = self.inputs["input_ids"].tolist()[0]

        if len(self.input_ids) > self.max_len:
            self.inputs = self.chunkify()
            self.chunked = True

    def chunkify(self):
        """ 
        Break up a long article into chunks that fit within the max token
        requirement for that Transformer model. 

        Calls to BERT / RoBERTa / ALBERT require the following format:
        [CLS] question tokens [SEP] context tokens [SEP].
        """

        # create question mask based on token_type_ids
        # value is 0 for question tokens, 1 for context tokens
        qmask = self.inputs['token_type_ids'].lt(1)
        qt = torch.masked_select(self.inputs['input_ids'], qmask)
        chunk_size = self.max_len - qt.size()[0] - 1 # the "-1" accounts for
        # having to add an ending [SEP] token to the end

        # create a dict of dicts; each sub-dict mimics the structure of pre-chunked model input
        chunked_input = OrderedDict()
        for k,v in self.inputs.items():
            q = torch.masked_select(v, qmask)
            c = torch.masked_select(v, ~qmask)
            chunks = torch.split(c, chunk_size)
            
            for i, chunk in enumerate(chunks):
                if i not in chunked_input:
                    chunked_input[i] = {}

                thing = torch.cat((q, chunk))
                if i != len(chunks)-1:
                    if k == 'input_ids':
                        thing = torch.cat((thing, torch.tensor([102])))
                    else:
                        thing = torch.cat((thing, torch.tensor([1])))

                chunked_input[i][k] = torch.unsqueeze(thing, dim=0)
        return chunked_input

    def get_answer(self,answer,src):
        if self.chunked:
            
            for k, chunk in self.inputs.items():
                answer_start_scores, answer_end_scores = self.model(**chunk)

                answer_start = torch.argmax(answer_start_scores)
                answer_end = torch.argmax(answer_end_scores) + 1
                ans = self.convert_ids_to_string(chunk['input_ids'][0][answer_start:answer_end])
                if ans.startswith(('[CLS]','[SEP]',' ','°')) or ans=='':
                    #raise Exception('No Answer')
                    pass
                else:
                    score = float(torch.max(answer_start_scores))
                    #print(ans,score)
                    answer.add_ans(ans,score,src)
                    #answer += ans + ', '
                    #print(ans,torch.max(answer_start_scores),torch.max(answer_end_scores))
                    #break
        else:
            answer_start_scores, answer_end_scores = self.model(**self.inputs)

            answer_start = torch.argmax(answer_start_scores)  # get the most likely beginning of answer with the argmax of the score
            answer_end = torch.argmax(answer_end_scores) + 1  # get the most likely end of answer with the argmax of the score
        
            ans = self.convert_ids_to_string(self.inputs['input_ids'][0][
                                              answer_start:answer_end])
            
            if ans in ['',' ','  ']:
              pass
            else:
              score = float(torch.max(answer_start_scores))
              #print(ans,score)
              answer.add_ans(ans,score,src)
        if len(answer) == 0:
          raise Error("No Answer") 
        return answer
        
    def convert_ids_to_string(self, input_ids):
        return self.tokenizer.convert_tokens_to_string(self.tokenizer.convert_ids_to_tokens(input_ids))

In [44]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import wikipedia as wiki
import pprint as pp
from collections import OrderedDict
import scipy
import requests
from bs4 import BeautifulSoup

class Korean_QA_on_Wiki:
  def __init__(self, document_reader,sentence_embedder):
    self.reader = document_reader
    self.embedder = sentence_embedder
    wiki.set_lang('ko')

  def __search_from_wiki(self,question,max_rank):
    results = wiki.search(question,results=max_rank)
    contents = []
    for result in results:
      try:
        page = wiki.page(result)
        #print(f"Top wiki result: {page}")
        text = page.content
        contents.append((text,page))
      except Exception as ex:
        print(ex)
    return contents

  def __search_from_naver(self,question,max_rank):
    contents = []
    url = 'https://search.naver.com/search.naver'
    for w in ['view','kin','news']:
      params = {'query': question,'where': w,}
      response = requests.get(url, params=params)
      html = response.text
      #뷰티풀소프의 인자값 지정
      soup = BeautifulSoup(html, 'html.parser')
      #쪼개기
      #title_list = soup.find_all('a', href=True)
      title_list = soup.select('.api_txt_lines')
      #print(title_list)
      tmp = []
      for tag in title_list:
        tmp.append(tag.text)
      contents.append((''.join(tmp),url + '?where='+w))
      tmp.clear()
    #print(contents)      
    return contents



  def question(self, questions, max_rank = 5):
    answers = {}
    for question in questions:
        print(f"Question: {question}")
        contents = []
        contents.extend(self.__search_from_wiki(question,max_rank))
        contents.extend(self.__search_from_naver(question,max_rank))
        print('-- Source count : ', len(contents))
        corpus_embeddings = self.embedder.encode([a for (a,b) in contents],show_progress_bar=False) 
        query_embeddings = self.embedder.encode([question])
        distances = scipy.spatial.distance.cdist(query_embeddings, corpus_embeddings, "cosine")[0]

        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])
        answer = AnswerVoter()
        for idx, distance in results:
            text = contents[idx][0]
            print('-- source :', contents[idx][1])
            try:
                self.reader.tokenize(question, text)
                self.reader.get_answer(answer,contents[idx][1])
                #answer.add_src(contents[idx][1])
                #t = (self.reader.get_answer(answer),contents[idx][1])
                #print(f"Answer: {t[0]}", f" from {t[1]}")
                #answer_list.append(t)
                
            except Exception as ex:
                pass    
        answers[question] = answer.get_ans()
        answer.print()
        print(' ')
    return answers


In [45]:
kqaw = Korean_QA_on_Wiki(DocumentReader(tokenizer,model),embedder)

In [6]:
answers = kqaw.question(["북한에서 실질적인 권력자는 누구인가?",
                           "세계에서 가장 넓은 호수는?",
                           "오로라가 가장 잘 보이는 곳은?",
                           "심장이 죄어오듯이 아프면 의심되는 병은 무엇인가?",
                           "항문에서 피가 나는 병은 무엇인가?",
                           "김재규는 박정희를 왜 죽였는가?",
                           "케네디를 죽인 암살범은 누구인가?",
                           "술 취하지 않는 방법은?",
                           "사람을 사랑해서 생기는 병은?",
                           "부모는 자식을 왜 사랑하는가?",
                           "나의 와이프는 나를 사랑하는가?",
                           "신은 존재 하는가?",
                           "사람의 인생에서 가장 소중한 것은 무엇인가?",
                           "바람난 여자는 다시 돌아올 수 있는가?",
                           "위가 쓰리고 아플 때 어떤 약을 복용해야 하는가?",
                           "눈알이 빠지면 어떻게 되는가?"])

Question: 북한에서 실질적인 권력자는 누구인가?
Answer: 누군가요 ? . . . 김정은은 그냥 얼굴마담인가요 ? 그럼 실질적인 북한권력의 주인이 누군가요 ? 김일성의 모습과 말투를 따라하는 김정은은 그냥 권력의 끝자락을 쥐고 있는 젊은이일 뿐인가요 ? 지금 북한의 권력구조에 대해 정통한 분 . . . 알려주세요 . . 북한의 실질적인 권력자가 누구인지 ? ? 언론 이나 관계기관의 발표 등을 종합해 볼 때 현재로서는 북한의 최고 권력자는 김정일의 대를 이은 김정은이라고 봐야 할 것입니다 . 김정은은 김정일 사망 후 권력을 인계받은 수 8개월동안 권력 안정화 작업을 진행중에 있으며 나름대로 성과를 거두고 있는것으로 알려지고 있습니다 . 그 . . . 6 . 25전쟁과 북한에 관한 건데요 . . . . 그리고 서울1945를 보면서 그 누구더라 그사람이 죽자 최운혁이 괴로워하면서 북으로 넘어갔는데 아 몽양 은 생각나는데 하여튼, 훅스, 원의,   from https://search.naver.com/search.naver?where=kin
Answer: 김정은, 김정일,   from https://search.naver.com/search.naver?where=news
Answer: 박정희, 박상희, 대통령,   from <WikipediaPage '박정희'>
Answer: 김정일, 장성택,   from <WikipediaPage '장성택'>
Answer: 이미, 김일성, 만주 빨치산 출신자들이, 김일성, 김용범, 김일성, 김일성, 김일성, 간혹,   from <WikipediaPage '김일성'>
Answer: 반드시 법적 책임을 져야 할 것 ” 이라고 경고했다 . 박지원 의원 측은 박 대표 개인 소셜미디어 계정에 “ 가짜뉴스 프레임으로 진실을 덮을 수 없음을 경고한다 ” 는 국민의당 논평을 공유했다 . 또한 국민의당 측은 4 . 17 오거돈 부산 상임선대위원장 발언 녹취록을 공개했다 . 이 녹취록에 대해 문 후보 선대위의 홍익표 수석대변인은 " 국민의

In [46]:
answers = kqaw.question(["아브라함은 자식이 몇명인가?"])

Question: 아브라함은 자식이 몇명인가?
-- Source count :  8
-- source : https://search.naver.com/search.naver?where=news
 --- Candidate answer: 몇 명 6.595310688018799
-- source : https://search.naver.com/search.naver?where=kin
 --- Candidate answer: 8명 11.673369407653809
-- source : <WikipediaPage '아브라함'>
 --- Candidate answer: 6명 6.828792095184326
-- source : <WikipediaPage '에티오피아 이름'>
-- source : https://search.naver.com/search.naver?where=view
-- source : <WikipediaPage '하와'>
 --- Candidate answer: 세 명의 11.601317405700684
-- source : <WikipediaPage '소저너 트루스'>
 --- Candidate answer: 13명 10.888596534729004
 --- Candidate answer: 다섯 명의 7.598954200744629
-- source : <WikipediaPage '믿음'>
Answer: 8명  score: 11.673369407653809  source: ['https://search.naver.com/search.naver?where=kin']
Answer: 세 명의  score: 11.601317405700684  source: [<WikipediaPage '하와'>]
 


In [47]:
answers = kqaw.question(["아시아나항공은 어디에 매각될 것인가?",
                         "박세창은 어느 회사의 사장인가?"])

Question: 아시아나항공은 어디에 매각될 것인가?
-- Source count :  4
-- source : https://search.naver.com/search.naver?where=kin
-- source : https://search.naver.com/search.naver?where=view
 --- Candidate answer: 아시아나 3.69708514213562
 --- Candidate answer: hdc 7.528763294219971
 --- Candidate answer: 금호리조트 8.839470863342285
-- source : https://search.naver.com/search.naver?where=news
 --- Candidate answer: 한진그룹에 8000억원을 투입하고 아시아나항공 매각을 추진하면서 양사 직원들은 큰 . . . 실제 만나고 싶다면 공문을 보내고 언제 어디서 누구와 어떤 안건으로 만나는 것인지 내용이 있어야 한다 . 그게 . . . 노조는 " 이상직 의원의 거액 매각대금 챙기기를 위해 코로나19를 빌미로 고용유지지원금 신청도 하지 않은 . . . 정부가 아시아나를 끌고 갈 것 ' 이라고 말했었다 . 그런데 보름도 안돼서 아시아나항공을 대한항공 8.913479804992676
 --- Candidate answer: 외부 컨설팅 9.669889450073242
-- source : <WikipediaPage '문화방송'>
 --- Candidate answer: LG 그룹 3.3898866176605225
Answer: 외부 컨설팅  score: 9.669889450073242  source: ['https://search.naver.com/search.naver?where=news']
Answer: 한진그룹에 8000억원을 투입하고 아시아나항공 매각을 추진하면서 양사 직원들은 큰 . . . 실제 만나고 싶다면 공문을 보내고 언제 어디서 누구와 어떤 안건으로 만나는 것인지 내용이 있어야 

In [48]:
answers = kqaw.question(["아시아나항공 사장의 이름은?",
                         "금호건설의 사장은 누구인가?"])

Question: 아시아나항공 사장의 이름은?
-- Source count :  8
-- source : https://search.naver.com/search.naver?where=view
 --- Candidate answer: 한창수 9.841452598571777
 --- Candidate answer: 김수천 11.469464302062988
-- source : <WikipediaPage '에어부산'>
-- source : <WikipediaPage '아시아나에어포트'>
 --- Candidate answer: 티웨이에어서비스 7.49818754196167
-- source : <WikipediaPage '박정구'>
 --- Candidate answer: 박정구 6.732430458068848
-- source : <WikipediaPage '인천국제공항'>
-- source : https://search.naver.com/search.naver?where=kin
 --- Candidate answer: 윤영두 11.365925788879395
-- source : <WikipediaPage '에어서울'>
-- source : https://search.naver.com/search.naver?where=news
 --- Candidate answer: 한창수 11.082985877990723
Answer: 한창수  score: 20.9244384765625  source: ['https://search.naver.com/search.naver?where=view', 'https://search.naver.com/search.naver?where=news']
Answer: 김수천  score: 11.469464302062988  source: ['https://search.naver.com/search.naver?where=view']
 
Question: 금호건설의 사장은 누구인가?
-- Source count :  4
-- source : <

In [ ]:
answers = kqaw.question(["아파트 값은 계속 오를 것인가?",
                         "코로나는 언제 종식 될 것인가?"])

In [ ]:
answers = kqaw.question(["단백질의 화학식 구성은 어떻게 되는가?"])

Question: 단백질의 화학식 구성은 어떻게 되는가?
13
<WikipediaPage '생화학'>
아미노산 tensor(2.1999, grad_fn=<MaxBackward1>)
최소 tensor(1.5361, grad_fn=<MaxBackward1>)
https://search.naver.com/search.naver?where=kin
노폐물 tensor(7.0935, grad_fn=<MaxBackward1>)
<WikipediaPage '단백질'>
NH2CHRnCOOH ) n tensor(9.9139, grad_fn=<MaxBackward1>)
<WikipediaPage '탄수화물'>
C5H10O4 tensor(4.5876, grad_fn=<MaxBackward1>)
C12H22O11 tensor(10.0584, grad_fn=<MaxBackward1>)
<WikipediaPage '분자'>
가끔 tensor(1.4926, grad_fn=<MaxBackward1>)
<WikipediaPage '아미노산'>
##NH2CHRnCOOH tensor(5.2119, grad_fn=<MaxBackward1>)
<WikipediaPage '다당류'>
C6H10O5 tensor(9.5896, grad_fn=<MaxBackward1>)
선형 구조 tensor(2.2327, grad_fn=<MaxBackward1>)
https://search.naver.com/search.naver?where=view
Geraniol tensor(6.4510, grad_fn=<MaxBackward1>)
<WikipediaPage '글라이신'>
HO2CCH2NH2 tensor(10.9986, grad_fn=<MaxBackward1>)
<WikipediaPage '세린'>
HO2CCH ( NH2 ) CH2OH tensor(6.9836, grad_fn=<MaxBackward1>)


In [49]:
answers = kqaw.question(["우리나라 특허의 권리보장 기간은 몇년인가?"])

Question: 우리나라 특허의 권리보장 기간은 몇년인가?
-- Source count :  8
-- source : https://search.naver.com/search.naver?where=kin
 --- Candidate answer: 20년 5.713622570037842
-- source : https://search.naver.com/search.naver?where=view
 --- Candidate answer: 1년 7.063043117523193
-- source : https://search.naver.com/search.naver?where=news
-- source : <WikipediaPage '민사소송법'>
-- source : <WikipediaPage '퍼블리시티권'>
-- source : <WikipediaPage '한국노동연구원'>
-- source : <WikipediaPage '대한민국'>
-- source : <WikipediaPage '유럽의 역사'>
Answer: 1년  score: 7.063043117523193  source: ['https://search.naver.com/search.naver?where=view']
Answer: 20년  score: 5.713622570037842  source: ['https://search.naver.com/search.naver?where=kin']
 


In [50]:
answers = kqaw.question(["발열 마른기침 피로감 등의 증상을 보이면 어떤 병이 의심되는가?"])

Question: 발열 마른기침 피로감 등의 증상을 보이면 어떤 병이 의심되는가?
-- Source count :  4
-- source : https://search.naver.com/search.naver?where=kin
 --- Candidate answer: 코로나 감염 6.755837440490723
 --- Candidate answer: 코로나 9.404107093811035
 --- Candidate answer: 코로나 19 6.107810974121094
-- source : https://search.naver.com/search.naver?where=news
-- source : https://search.naver.com/search.naver?where=view
 --- Candidate answer: 코로나19 3.9041125774383545
 --- Candidate answer: 코로나 7.809553623199463
-- source : <WikipediaPage '코로나19 범유행'>
 --- Candidate answer: 호흡곤란 , 후각상실 , 미각상실 6.361567974090576
 --- Candidate answer: 경증에서 중증의 증상을 보일 수 있다 . 또한 무증상 3.518268346786499
 --- Candidate answer: 두통 , 호흡곤란 , 폐렴 5.779277324676514
Answer: 코로나  score: 17.213660717010498  source: ['https://search.naver.com/search.naver?where=kin', 'https://search.naver.com/search.naver?where=view']
Answer: 코로나 19  score: 10.011923551559448  source: ['https://search.naver.com/search.naver?where=kin', 'https://search.naver.com/search.na

In [51]:
answers = kqaw.question(["흉부통증과 호흡곤란, 쉰목소리, 가끔 피가 썩인 가래도 있습니다. 어떤 병일까요?"])

Question: 흉부통증과 호흡곤란, 쉰목소리, 가끔 피가 썩인 가래도 있습니다. 어떤 병일까요?
-- Source count :  3
-- source : https://search.naver.com/search.naver?where=view
 --- Candidate answer: 폐암 8.353275299072266
-- source : https://search.naver.com/search.naver?where=kin
-- source : https://search.naver.com/search.naver?where=news
Answer: 폐암  score: 8.353275299072266  source: ['https://search.naver.com/search.naver?where=view']
 


In [52]:
answers = kqaw.question(["똥을 싸고 나면 휴지에 피가 뭍습니다. 의심되는 병은 무엇인가요?"])

Question: 똥을 싸고 나면 휴지에 피가 뭍습니다. 의심되는 병은 무엇인가요?
-- Source count :  3
-- source : https://search.naver.com/search.naver?where=kin
 --- Candidate answer: 치열 8.134779930114746
 --- Candidate answer: 치열 8.31339168548584
 --- Candidate answer: 궤양성 대장염 9.893392562866211
-- source : https://search.naver.com/search.naver?where=view
 --- Candidate answer: 호흡기질환 어떤 것이 있을까 ? 사람의 생명을 유지하는데 있어 기본이 되는 것은 무엇인가라고 묻는다면 음식 , 잠 등 정말 다양한 의견들이 . . . 아닌지 의심해 볼 필요가 있습니다 물론 단순한 감기 정도야 시간이 지나면 금방 좋아집니다만 자칫 이러한 . . . ③생쟝에서 론세스바예스 : 피레네를 [UNK] 가방과 스틱을 내팽개치고 휴지를 꺼내 풀숲으로 뛰었다 . 지나가는 이들이 쳐다보면 다 . . . 뒤에서 누군가 온다면 ' 나 똥 싸고 있어요 ' 라며 얼굴로 인사할 수밖에 없는 위치 , 그곳에서 나는 해결할 . . . 귀에서 진물이 나요 - 성수이비인후과의 중이염치료귀에서 진물이 나요 귀에서 진물이 나면 외이도염과 중이염 5.404848575592041
 --- Candidate answer: 혈변 7.531818866729736
-- source : https://search.naver.com/search.naver?where=news
Answer: 치열  score: 16.448171615600586  source: ['https://search.naver.com/search.naver?where=kin']
Answer: 궤양성 대장염  score: 9.893392562866211  source: ['https://search.naver.c

In [ ]:
answers = kqaw.question(["신경망 알고리즘의 활성함수에는 어떤 것이 있나요?"])

In [ ]:
answers = kqaw.question(["현존하는 인공지능 중 가장 성능이 우수한 것은 무엇입니까?"])

In [53]:
answers = kqaw.question(["이세돌을 이긴 것은 무엇입니까?"])

Question: 이세돌을 이긴 것은 무엇입니까?
-- Source count :  7
-- source : https://search.naver.com/search.naver?where=view
 --- Candidate answer: 알파고 4.502975940704346
-- source : https://search.naver.com/search.naver?where=news
-- source : <WikipediaPage '다큐플렉스의 에피소드 목록'>
-- source : <WikipediaPage '인공지능'>
 --- Candidate answer: 깔끔함 8.735248565673828
 --- Candidate answer: 알파고 11.076563835144043
-- source : https://search.naver.com/search.naver?where=kin
 --- Candidate answer: 알파고 10.360079765319824
 --- Candidate answer: 알파고 11.44482421875
 --- Candidate answer: 알파고 4.098506450653076
 --- Candidate answer: 알파고 9.944950103759766
-- source : <WikipediaPage '알파고'>
 --- Candidate answer: 알파고 9.304676055908203
-- source : <WikipediaPage '아침마당의 에피소드 목록'>
Answer: 알파고  score: 60.73257637023926  source: ['https://search.naver.com/search.naver?where=view', <WikipediaPage '인공지능'>, 'https://search.naver.com/search.naver?where=kin', <WikipediaPage '알파고'>]
Answer: 깔끔함  score: 8.735248565673828  source: [<Wikip

In [54]:
answers = kqaw.question(["피타고라스는 어느 나라 사람인가?"])

Question: 피타고라스는 어느 나라 사람인가?
-- Source count :  8
-- source : https://search.naver.com/search.naver?where=kin
 --- Candidate answer: 그리스 8.628893852233887
 --- Candidate answer: 그리스 6.524637222290039
-- source : <WikipediaPage '고대 그리스 철학'>
 --- Candidate answer: 이탈리아 4.867263317108154
-- source : https://search.naver.com/search.naver?where=view
 --- Candidate answer: 호주 5.215020656585693
 --- Candidate answer: 그리스 6.704745292663574
 --- Candidate answer: 영국 4.360252380371094
 --- Candidate answer: 이집트 6.741414546966553
-- source : https://search.naver.com/search.naver?where=news
 --- Candidate answer: 그리스 5.066105365753174
-- source : <WikipediaPage '화신'>
-- source : <WikipediaPage '3'>
-- source : <WikipediaPage '수학자'>
 --- Candidate answer: 그리스 8.3052396774292
-- source : <WikipediaPage '종교'>
Answer: 그리스  score: 35.22962141036987  source: ['https://search.naver.com/search.naver?where=kin', 'https://search.naver.com/search.naver?where=view', 'https://search.naver.com/search.naver?wher

In [55]:
answers = kqaw.question(["아이즈원 멤버 인원수는?"])

Question: 아이즈원 멤버 인원수는?
-- Source count :  5
-- source : https://search.naver.com/search.naver?where=news
 --- Candidate answer: 12명 11.849099159240723
 --- Candidate answer: 10인조 6.792560577392578
-- source : https://search.naver.com/search.naver?where=view
 --- Candidate answer: 6명 11.00613784790039
 --- Candidate answer: 12 8.787215232849121
-- source : https://search.naver.com/search.naver?where=kin
 --- Candidate answer: 8명 10.086480140686035
 --- Candidate answer: 4명 6.36948823928833
-- source : <WikipediaPage 'HKT48'>
 --- Candidate answer: 300명 11.039990425109863
-- source : <WikipediaPage '프리큐어 시리즈'>
Answer: 12명  score: 11.849099159240723  source: ['https://search.naver.com/search.naver?where=news']
Answer: 300명  score: 11.039990425109863  source: [<WikipediaPage 'HKT48'>]
 


In [56]:
answers = kqaw.question(["트와이스 중에 가장 인기 있는 사람은?"])

Question: 트와이스 중에 가장 인기 있는 사람은?
-- Source count :  8
-- source : https://search.naver.com/search.naver?where=kin
 --- Candidate answer: 아베 나츠미 10.026005744934082
 --- Candidate answer: 유덕화 7.9194207191467285
-- source : https://search.naver.com/search.naver?where=view
 --- Candidate answer: 사나 4.985182762145996
 --- Candidate answer: 바이너리옵션 8.480669975280762
-- source : https://search.naver.com/search.naver?where=news
 --- Candidate answer: BTS 3.5792059898376465
 --- Candidate answer: 지효 10.526061058044434
-- source : <WikipediaPage '박진영'>
 --- Candidate answer: 박진영 9.716397285461426
-- source : <WikipediaPage '지효'>
-- source : <WikipediaPage '청하 (가수)'>
 --- Candidate answer: 지효 5.938498497009277
-- source : <WikipediaPage '트와이스'>
-- source : <WikipediaPage '사나 (가수)'>
Answer: 지효  score: 16.46455955505371  source: ['https://search.naver.com/search.naver?where=news', <WikipediaPage '청하 (가수)'>]
Answer: 아베 나츠미  score: 10.026005744934082  source: ['https://search.naver.com/search.naver?whe

In [57]:
answers = kqaw.question(["벤츠 자동차가 처음 발명된 년도는?"])

Question: 벤츠 자동차가 처음 발명된 년도는?
-- Source count :  8
-- source : https://search.naver.com/search.naver?where=kin
 --- Candidate answer: 1883년 11.766658782958984
 --- Candidate answer: 1886년 9.813643455505371
 --- Candidate answer: 1998년 4.83826208114624
-- source : <WikipediaPage '지크프리트 마르쿠스'>
 --- Candidate answer: 1831년 6.848933696746826
-- source : https://search.naver.com/search.naver?where=view
 --- Candidate answer: 1885년 6.031770706176758
 --- Candidate answer: 1887년 10.240007400512695
-- source : <WikipediaPage '자동차'>
 --- Candidate answer: 1886년 4.784786701202393
 --- Candidate answer: 1930년 5.459348201751709
-- source : <WikipediaPage '만하임'>
 --- Candidate answer: 1886년 11.38278865814209
-- source : <WikipediaPage '버스'>
 --- Candidate answer: 1893년 9.832870483398438
-- source : <WikipediaPage '리눅스'>
 --- Candidate answer: 1990년 5.884979248046875
-- source : https://search.naver.com/search.naver?where=news
Answer: 1886년  score: 25.981218814849854  source: ['https://search.naver.

In [ ]:
answers = kqaw.question(["교통사고 대비를 위해 들어야 하는 보험은 무엇인가?"])

In [ ]:
answers = kqaw.question(["너의 이름은 무엇이니?"])

In [ ]:
answers = kqaw.question(["너는 남자니 여자니?"])